<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Input/Output-Layers" data-toc-modified-id="Input/Output-Layers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Input/Output Layers</a></span><ul class="toc-item"><li><span><a href="#Input" data-toc-modified-id="Input-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Input</a></span></li><li><span><a href="#Output" data-toc-modified-id="Output-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Output</a></span></li></ul></li><li><span><a href="#Core-Layers" data-toc-modified-id="Core-Layers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Core Layers</a></span><ul class="toc-item"><li><span><a href="#Activation" data-toc-modified-id="Activation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Activation</a></span></li><li><span><a href="#ActivityRegularization" data-toc-modified-id="ActivityRegularization-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ActivityRegularization</a></span></li><li><span><a href="#Dense" data-toc-modified-id="Dense-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Dense</a></span></li><li><span><a href="#Dropout" data-toc-modified-id="Dropout-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Dropout</a></span></li><li><span><a href="#Flatten" data-toc-modified-id="Flatten-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Flatten</a></span></li><li><span><a href="#Highway" data-toc-modified-id="Highway-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Highway</a></span></li><li><span><a href="#Lambda" data-toc-modified-id="Lambda-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Lambda</a></span></li><li><span><a href="#Masking" data-toc-modified-id="Masking-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Masking</a></span></li><li><span><a href="#MaxoutDense" data-toc-modified-id="MaxoutDense-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>MaxoutDense</a></span></li><li><span><a href="#Permute" data-toc-modified-id="Permute-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>Permute</a></span></li><li><span><a href="#RepeatVector" data-toc-modified-id="RepeatVector-2.11"><span class="toc-item-num">2.11&nbsp;&nbsp;</span>RepeatVector</a></span></li><li><span><a href="#Reshape" data-toc-modified-id="Reshape-2.12"><span class="toc-item-num">2.12&nbsp;&nbsp;</span>Reshape</a></span></li><li><span><a href="#SpatialDropout1D" data-toc-modified-id="SpatialDropout1D-2.13"><span class="toc-item-num">2.13&nbsp;&nbsp;</span>SpatialDropout1D</a></span></li><li><span><a href="#SpatialDropout2D" data-toc-modified-id="SpatialDropout2D-2.14"><span class="toc-item-num">2.14&nbsp;&nbsp;</span>SpatialDropout2D</a></span></li><li><span><a href="#SpatialDropout3D" data-toc-modified-id="SpatialDropout3D-2.15"><span class="toc-item-num">2.15&nbsp;&nbsp;</span>SpatialDropout3D</a></span></li><li><span><a href="#TimeDistributedDense" data-toc-modified-id="TimeDistributedDense-2.16"><span class="toc-item-num">2.16&nbsp;&nbsp;</span>TimeDistributedDense</a></span></li></ul></li><li><span><a href="#Convolutional-Layers" data-toc-modified-id="Convolutional-Layers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Convolutional Layers</a></span><ul class="toc-item"><li><span><a href="#AtrousConvolution1D" data-toc-modified-id="AtrousConvolution1D-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>AtrousConvolution1D</a></span></li><li><span><a href="#AtrousConvolution2D" data-toc-modified-id="AtrousConvolution2D-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>AtrousConvolution2D</a></span></li><li><span><a href="#Convolution1D" data-toc-modified-id="Convolution1D-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Convolution1D</a></span></li><li><span><a href="#Convolution2D" data-toc-modified-id="Convolution2D-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Convolution2D</a></span></li><li><span><a href="#Convolution3D" data-toc-modified-id="Convolution3D-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Convolution3D</a></span></li><li><span><a href="#Cropping1D" data-toc-modified-id="Cropping1D-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Cropping1D</a></span></li><li><span><a href="#Cropping2D" data-toc-modified-id="Cropping2D-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Cropping2D</a></span></li><li><span><a href="#Cropping3D" data-toc-modified-id="Cropping3D-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Cropping3D</a></span></li><li><span><a href="#Deconvolution2D" data-toc-modified-id="Deconvolution2D-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>Deconvolution2D</a></span></li><li><span><a href="#SeparableConvolution2D" data-toc-modified-id="SeparableConvolution2D-3.10"><span class="toc-item-num">3.10&nbsp;&nbsp;</span>SeparableConvolution2D</a></span></li><li><span><a href="#UpSampling1D" data-toc-modified-id="UpSampling1D-3.11"><span class="toc-item-num">3.11&nbsp;&nbsp;</span>UpSampling1D</a></span></li><li><span><a href="#UpSampling2D" data-toc-modified-id="UpSampling2D-3.12"><span class="toc-item-num">3.12&nbsp;&nbsp;</span>UpSampling2D</a></span></li><li><span><a href="#UpSampling3D" data-toc-modified-id="UpSampling3D-3.13"><span class="toc-item-num">3.13&nbsp;&nbsp;</span>UpSampling3D</a></span></li><li><span><a href="#ZeroPadding1D" data-toc-modified-id="ZeroPadding1D-3.14"><span class="toc-item-num">3.14&nbsp;&nbsp;</span>ZeroPadding1D</a></span></li><li><span><a href="#ZeroPadding2D" data-toc-modified-id="ZeroPadding2D-3.15"><span class="toc-item-num">3.15&nbsp;&nbsp;</span>ZeroPadding2D</a></span></li><li><span><a href="#ZeroPadding3D" data-toc-modified-id="ZeroPadding3D-3.16"><span class="toc-item-num">3.16&nbsp;&nbsp;</span>ZeroPadding3D</a></span></li></ul></li><li><span><a href="#Pooling-Layers" data-toc-modified-id="Pooling-Layers-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pooling Layers</a></span><ul class="toc-item"><li><span><a href="#AvgPooling1D" data-toc-modified-id="AvgPooling1D-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>AvgPooling1D</a></span></li><li><span><a href="#AvgPooling2D" data-toc-modified-id="AvgPooling2D-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>AvgPooling2D</a></span></li><li><span><a href="#AvgPooling3D" data-toc-modified-id="AvgPooling3D-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>AvgPooling3D</a></span></li><li><span><a href="#MaxPooling1D" data-toc-modified-id="MaxPooling1D-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>MaxPooling1D</a></span></li><li><span><a href="#MaxPooling2D" data-toc-modified-id="MaxPooling2D-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>MaxPooling2D</a></span></li><li><span><a href="#MaxPooling3D" data-toc-modified-id="MaxPooling3D-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>MaxPooling3D</a></span></li><li><span><a href="#GlobalAvgPooling1D" data-toc-modified-id="GlobalAvgPooling1D-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>GlobalAvgPooling1D</a></span></li><li><span><a href="#GlobalAvgPooling2D" data-toc-modified-id="GlobalAvgPooling2D-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>GlobalAvgPooling2D</a></span></li><li><span><a href="#GlobalAvgPooling3D" data-toc-modified-id="GlobalAvgPooling3D-4.9"><span class="toc-item-num">4.9&nbsp;&nbsp;</span>GlobalAvgPooling3D</a></span></li><li><span><a href="#GlobalMaxPooling1D" data-toc-modified-id="GlobalMaxPooling1D-4.10"><span class="toc-item-num">4.10&nbsp;&nbsp;</span>GlobalMaxPooling1D</a></span></li><li><span><a href="#GlobalMaxPooling2D" data-toc-modified-id="GlobalMaxPooling2D-4.11"><span class="toc-item-num">4.11&nbsp;&nbsp;</span>GlobalMaxPooling2D</a></span></li><li><span><a href="#GlobalMaxPooling3D" data-toc-modified-id="GlobalMaxPooling3D-4.12"><span class="toc-item-num">4.12&nbsp;&nbsp;</span>GlobalMaxPooling3D</a></span></li></ul></li><li><span><a href="#Local-Layers" data-toc-modified-id="Local-Layers-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Local Layers</a></span><ul class="toc-item"><li><span><a href="#LocallyConnected1D" data-toc-modified-id="LocallyConnected1D-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>LocallyConnected1D</a></span></li><li><span><a href="#LocallyConnected2D" data-toc-modified-id="LocallyConnected2D-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>LocallyConnected2D</a></span></li></ul></li><li><span><a href="#Recurrent-Layers" data-toc-modified-id="Recurrent-Layers-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Recurrent Layers</a></span><ul class="toc-item"><li><span><a href="#GRU" data-toc-modified-id="GRU-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>GRU</a></span></li><li><span><a href="#LSTM" data-toc-modified-id="LSTM-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>LSTM</a></span></li><li><span><a href="#SimpleRNN" data-toc-modified-id="SimpleRNN-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>SimpleRNN</a></span></li></ul></li><li><span><a href="#Embeddings-Layers" data-toc-modified-id="Embeddings-Layers-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Embeddings Layers</a></span><ul class="toc-item"><li><span><a href="#Embeding" data-toc-modified-id="Embeding-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Embeding</a></span></li></ul></li><li><span><a href="#Convolutional-Recurrent-Layers" data-toc-modified-id="Convolutional-Recurrent-Layers-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Convolutional Recurrent Layers</a></span><ul class="toc-item"><li><span><a href="#ConvLSTM2D" data-toc-modified-id="ConvLSTM2D-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>ConvLSTM2D</a></span></li><li><span><a href="#ConvRecurrent2D" data-toc-modified-id="ConvRecurrent2D-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>ConvRecurrent2D</a></span></li></ul></li><li><span><a href="#Advanced-Activations-Layers" data-toc-modified-id="Advanced-Activations-Layers-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Advanced Activations Layers</a></span><ul class="toc-item"><li><span><a href="#ELU" data-toc-modified-id="ELU-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>ELU</a></span></li><li><span><a href="#LeakyReLU" data-toc-modified-id="LeakyReLU-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>LeakyReLU</a></span></li><li><span><a href="#ParametricSoftplus" data-toc-modified-id="ParametricSoftplus-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>ParametricSoftplus</a></span></li><li><span><a href="#PReLU" data-toc-modified-id="PReLU-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>PReLU</a></span></li><li><span><a href="#SReLU" data-toc-modified-id="SReLU-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>SReLU</a></span></li><li><span><a href="#ThreasholdedReLU" data-toc-modified-id="ThreasholdedReLU-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span>ThreasholdedReLU</a></span></li></ul></li><li><span><a href="#Normalization-Layers" data-toc-modified-id="Normalization-Layers-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Normalization Layers</a></span><ul class="toc-item"><li><span><a href="#BatchNormalization" data-toc-modified-id="BatchNormalization-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>BatchNormalization</a></span></li></ul></li><li><span><a href="#Noise-Layers" data-toc-modified-id="Noise-Layers-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Noise Layers</a></span><ul class="toc-item"><li><span><a href="#GaussianDropout" data-toc-modified-id="GaussianDropout-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>GaussianDropout</a></span></li><li><span><a href="#GaussianNoise" data-toc-modified-id="GaussianNoise-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>GaussianNoise</a></span></li></ul></li><li><span><a href="#Special-Functions" data-toc-modified-id="Special-Functions-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Special Functions</a></span><ul class="toc-item"><li><span><a href="#Merge" data-toc-modified-id="Merge-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Merge</a></span></li></ul></li><li><span><a href="#Pre-Trained-Models" data-toc-modified-id="Pre-Trained-Models-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Pre-Trained Models</a></span><ul class="toc-item"><li><span><a href="#InceptionV3" data-toc-modified-id="InceptionV3-13.1"><span class="toc-item-num">13.1&nbsp;&nbsp;</span>InceptionV3</a></span></li><li><span><a href="#ResNet50" data-toc-modified-id="ResNet50-13.2"><span class="toc-item-num">13.2&nbsp;&nbsp;</span>ResNet50</a></span></li><li><span><a href="#SqueezeNet" data-toc-modified-id="SqueezeNet-13.3"><span class="toc-item-num">13.3&nbsp;&nbsp;</span>SqueezeNet</a></span></li><li><span><a href="#VGG16" data-toc-modified-id="VGG16-13.4"><span class="toc-item-num">13.4&nbsp;&nbsp;</span>VGG16</a></span></li><li><span><a href="#VGG19" data-toc-modified-id="VGG19-13.5"><span class="toc-item-num">13.5&nbsp;&nbsp;</span>VGG19</a></span></li><li><span><a href="#WideResNet" data-toc-modified-id="WideResNet-13.6"><span class="toc-item-num">13.6&nbsp;&nbsp;</span>WideResNet</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>References</a></span><ul class="toc-item"><li><span><a href="#deepcognition" data-toc-modified-id="deepcognition-14.1"><span class="toc-item-num">14.1&nbsp;&nbsp;</span><a href="https://app.deepcognition.ai" target="_blank">deepcognition</a></a></span></li></ul></li></ul></div>

# <div align='center'> 拖拽模型 </div>

## Input/Output Layers

###  Input

### Output

## Core Layers

###  Activation

###  ActivityRegularization

### Dense

### Dropout

### Flatten

### Highway

### Lambda

### Masking

### MaxoutDense

### Permute

### RepeatVector

### Reshape

### SpatialDropout1D

### SpatialDropout2D

### SpatialDropout3D

### TimeDistributedDense

## Convolutional Layers

### AtrousConvolution1D

### AtrousConvolution2D

### Convolution1D

### Convolution2D

### Convolution3D

### Cropping1D

### Cropping2D

### Cropping3D

### Deconvolution2D

### SeparableConvolution2D

### UpSampling1D

### UpSampling2D

### UpSampling3D

### ZeroPadding1D

### ZeroPadding2D

### ZeroPadding3D

## Pooling Layers

### AvgPooling1D

### AvgPooling2D

### AvgPooling3D

### MaxPooling1D

### MaxPooling2D

### MaxPooling3D

### GlobalAvgPooling1D

### GlobalAvgPooling2D

### GlobalAvgPooling3D

### GlobalMaxPooling1D

### GlobalMaxPooling2D

### GlobalMaxPooling3D

## Local Layers

### LocallyConnected1D

### LocallyConnected2D

## Recurrent Layers

### GRU

### LSTM

### SimpleRNN

## Embeddings Layers

### Embeding

## Convolutional Recurrent Layers

### ConvLSTM2D

### ConvRecurrent2D

## Advanced Activations Layers

### ELU

### LeakyReLU

### ParametricSoftplus

### PReLU

### SReLU

### ThreasholdedReLU

## Normalization Layers

### BatchNormalization

## Noise Layers

### GaussianDropout

### GaussianNoise

## Special Functions

### Merge

## Pre-Trained Models

### InceptionV3

### ResNet50

### SqueezeNet

### VGG16

### VGG19

### WideResNet

## References

### [deepcognition](https://app.deepcognition.ai)